<a href="https://colab.research.google.com/github/KevinTheRainmaker/MLOps/blob/main/MLOps_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#데이터 전처리 using TFT

In [1]:
!pip install -q tensorflow-transform

     |████████████████████████████████| 422 kB 17.8 MB/s 
     |████████████████████████████████| 48 kB 3.5 MB/s 
     |████████████████████████████████| 9.9 MB 40.7 MB/s 
     |████████████████████████████████| 19.1 MB 19.9 MB/s 
     |████████████████████████████████| 247 kB 35.8 MB/s 
     |████████████████████████████████| 151 kB 47.0 MB/s 
     |████████████████████████████████| 2.3 MB 57.6 MB/s 
     |████████████████████████████████| 45 kB 3.0 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 83 kB 1.7 MB/s 
     |████████████████████████████████| 173 kB 78.2 MB/s 
     |████████████████████████████████| 255 kB 55.1 MB/s 
     |████████████████████████████████| 171 kB 64.6 MB/s 
     |████████████████████████████████| 180 kB 59.0 MB/s 
     |████████████████████████████████| 183 kB 63.0 MB/s 
     |████████████████████████████████| 144 kB 51.6 MB/s 
     |████████████████████████████████| 110 kB 45.8 MB/s 
     |███████████████

In [2]:
import tensorflow_transform as tft

def preprocessing_fn(inputs):
  x = inputs['x']
  x_normalized = tft.scale_to_0_1(x)

  return {
      'x_xf': x_normalized
  }

In [3]:
import tensorflow as tf

def preprocessing_fn(raw_image):
  raw_image = tf.reshape(raw_image, [-1])

  # JPEG Decoding
  img_rgb = tf.io.decode_jpeg(raw_image, channels=3)

  # RGB to Gray
  img_gray = tf.image.rgb_to_grayscale(img_rgb)
  img = tf.image.convert_image_dtype(img_gray, tf.float32)

  # Re-sizing the image to 300*300
  resized_img = tf.image.resize_with_pad(
      img,
      target_height=300,
      target_width=300
  )

  img_grayscale = tf.image.rgb_to_grayscale(resized_img)

  return tf.reshape(img_grayscale, [-1, 300, 300, 1])

In [5]:
raw_data = [
            {'x':   1.20},
            {'x':   2.99},
            {'x': 100.00}
]

In [6]:
import tensorflow as tf
from tensorflow_transform.tf_metadata import dataset_metadata
from tensorflow_transform.tf_metadata import schema_utils

raw_data_metadata = dataset_metadata.DatasetMetadata(
    schema_utils.schema_from_feature_spec({
        'x':tf.io.FixedLenFeature([], tf.float32),
    })
)

In [ ]:
import tempfile
import tensorflow_transform.beam.impl as tft_beam

# 이전에 정의한 preprocessing_fn()
def preprocessing_fn(inputs):
  x = inputs['x']
  x_normalized = tft.scale_to_0_1(x)

  return {
      'x_xf': x_normalized
  }

with beam.Pipeline() as pipeline:
  with tft_beam.Context(temp_dir=tempfile.mkdtemp()):

    tfrecord_file = 'TFRecords_File.tfrecord'
    raw_data = (
        pipeline|beam.io.ReadFromTFRecord(tfrecord_file)
    )

    transformed_dataset, transform_fn = (
        (raw_data, raw_data_metadata)|tft_beam.AnalyzeAndTransformDataset(preprocessing_fn)
    )

In [ ]:
transformed_data, transformed_metadata = transformed_dataset

print(transformed_data)

In [7]:
# [
#  {'x_xf': 0.0},
#  {'x_xf': 0.018117407},
#  {'x_xf': 1.0}
# ]